In [2]:
from helpers import *
import pandas as pd 
import numpy as np 
import re
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="user_name_agent")

import time

In [3]:
#Realizar Web Scraping 
prices,address,m2,rooms,toilets,urls = scrapingportal_inmobiliario(region="santiago",pages=10)

Running Web Scraping Portal Inmobiliario...
prices: +44 + attributes: +44  address: +44
prices: +93 + attributes: +93  address: +93
prices: +143 + attributes: +143  address: +143
prices: +191 + attributes: +191  address: +191
prices: +241 + attributes: +241  address: +241
prices: +289 + attributes: +289  address: +289
prices: +338 + attributes: +338  address: +338
prices: +386 + attributes: +386  address: +386
prices: +436 + attributes: +436  address: +436
prices: +483 + attributes: +483  address: +483
Webscraping Finished



In [4]:
df = pd.DataFrame({'urls':urls,'address':address,'m2':m2,'rooms':rooms,'toilets':toilets,'prices':prices})

In [5]:
#Modificar  valores almacenados  en  columnas no correspondientes
cond1 = df.rooms.str.contains(fr"baño[s]",regex=True)
cond2 = df.m2.str.contains(fr"dormitorio[s]",regex=True)
cond3 = df.m2.str.contains(fr"baño[s]",regex=True)

df['toilets'] = np.where(cond1,df.rooms,df.toilets)
df["rooms"] = np.where(cond2,df.m2,df.rooms)
df['toilets'] = np.where(cond3,df.m2,df.toilets)

#Eliminar columnas que tienen valores nulos en mas de 1 columna (Toilets y Rooms en este caso)
df = df.drop(df[df.rooms.isna()].index)
#df = df.drop(65)

#Extraer  datos numerics de columnas
df.rooms = df.rooms.str.extract(r"(\d)")
df.m2=df.m2.str.extract(r"(\d+)")
df.toilets=df.toilets.str.extract(r"(\d+)")
df.prices = df.prices.str.replace(".","")

#Cambiar tipo de datos a numerico
df.m2 = df.m2.astype("int")
df.rooms = df.rooms.astype("int")
df.toilets = df.toilets.astype("float")
df.prices = df.prices.astype("int")

#Eliminar Filas que tengan precios menores a 100.000 o mayores a 10.000.000
df = df.drop(df[df.prices<100000].index)
df = df.drop(df[df.prices>10000000].index)

#Reemplazar con la media valor de baño
df.toilets.fillna(df.toilets.mean(),inplace=True)

#volver a reemplazar toilepor int
df.toilets = df.toilets.astype("int")

#Resetear index del  Data Frame
df = df.reset_index(drop=True)


In [47]:
#Agregar coordenadas geograficas de direcciones a Data Frame con libreria geopy
targets_address= df['address'].tolist()

latitude = []
longitude = []

for target in targets_address:

    location = geolocator.geocode(target)
    if location == None:
        latitude.append(np.nan)
        longitude.append(np.nan)
    else:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    time.sleep(0.2)

In [48]:
#Almacenar resultados en Data Frame
df['latitude'] = latitude
df['longitude'] = longitude

In [49]:
#Crear nuevo data frame con  direcciones validas y coordeandas geograficas no nulas 
geo_df = df[~df['latitude'].isnull()]

In [50]:
geo_df

,urls,address,m2,rooms,toilets,prices,latitude,longitude
1,https://www.portalinmobiliario.com/MLC-5414693...,"Mujica 99, Ñuñoa, Chile, Ñuñoa",40,1,1,320000,-33.460586,-70.605999
2,https://www.portalinmobiliario.com/MLC-5415290...,"Mario Kreutzberger 1558, Santiago, Chile, San...",33,1,1,235000,-33.435944,-70.659488
3,https://www.portalinmobiliario.com/MLC-5411404...,"José Domingo Cañas 705, Ñuñoa, Chile, Ñuñoa",65,2,2,490000,-33.456710,-70.609417
4,https://www.portalinmobiliario.com/MLC-5414400...,"Irarrázaval 200, Ñuñoa, Chile, Ñuñoa",24,1,1,300000,-33.454189,-70.604972
5,https://www.portalinmobiliario.com/MLC-5415165...,"Avenida España 199, Santiago, Chile, Santiago",35,1,1,260000,-33.451430,-70.669938


In [51]:
geo_df.to_csv("data-deptos.csv")